In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [2]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


### loading Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
with open("/content/drive/My Drive/Files/network_input.pkl", 'rb') as f:
    network_input = pickle.load(f)
with open("/content/drive/My Drive/Files/network_output.pkl", 'rb') as f:
    network_output = pickle.load(f)
with open("/content/drive/My Drive/Files/notes.pkl", 'rb') as f:
    notes= pickle.load(f)

In [5]:
n_vocab = len(set(notes))

## Create Model

In [6]:
""" create the structure of the neural network """
model = Sequential()
model.add(LSTM(512,
            input_shape=(network_input.shape[1], network_input.shape[2]),
            return_sequences=True ))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(optimizer="adamax", loss='categorical_crossentropy')

In [7]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)              

In [ ]:
""" train the neural network """

filepath = "/content/drive/My Drive/model/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

checkpoint = ModelCheckpoint(
    filepath,
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min')

callbacks_list = [checkpoint]

model_his = model.fit(network_input, network_output, epochs=100, batch_size=64, callbacks=callbacks_list)

Epoch 1/100
60398/60398 [==============================] - 760s 13ms/step - loss: 4.7168
Epoch 2/100
60398/60398 [==============================] - 775s 13ms/step - loss: 4.5663
Epoch 3/100
60398/60398 [==============================] - 775s 13ms/step - loss: 4.5092
Epoch 4/100
60398/60398 [==============================] - 769s 13ms/step - loss: 4.4492
Epoch 5/100
60398/60398 [==============================] - 768s 13ms/step - loss: 4.3424
Epoch 6/100
60398/60398 [==============================] - 775s 13ms/step - loss: 4.2173
Epoch 7/100
60398/60398 [==============================] - 773s 13ms/step - loss: 4.0821
Epoch 8/100
60398/60398 [==============================] - 772s 13ms/step - loss: 3.9045
Epoch 9/100
60398/60398 [==============================] - 775s 13ms/step - loss: 3.7196
Epoch 10/100
60398/60398 [==============================] - 784s 13ms/step - loss: 3.5320
Epoch 11/100
60398/60398 [==============================] - 774s 13ms/step - loss: 3.3437
Epoch 12/100
60398/